# Random Forest Interpretation

WNixalo 2018/5/13

Codealong of [fast.ai ML1 L3](https://youtu.be/YSFG_W8JxBo): [notebook](https://github.com/fastai/fastai/blob/master/courses/ml1/lesson2-rf_interpretation.ipynb)

---

> [lecture 00:01:33](https://youtu.be/YSFG_W8JxBo?t=93) Random forests allow us to understand our data deeper and more quickly than traditional approaches.

Q: "when should we use Random Forests?"

A: "the question is more: *when should I use other things as well?"

> for Unstructured Data (sound waveform, words in text, pixels in images) I'll almost certainly want to try Deep Learning. For Collaborative Filtering, neither DL nor RF are exactly what you want (tweaks are needed)

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics

In [4]:
set_plot_sizes(12,14,16)

## 1. Load in our data from last lesson

In [6]:
PATH = "data/bulldozers/"
df_raw = pd.read_feather('tmp/raw')
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice')

ArrowIOError: Failed to open local file: tmp/raw , error: No such file or directory